In [1]:
# %load_ext autoreload
# %autoreload 2
%load_ext lab_black

import os
import numpy as np
import pandas as pd
from glob import glob
import statsmodels.api as sm
from typing import List
import itertools
import matplotlib.pyplot as plt
from scipy import stats
import calpgs
import seaborn as sns
from tqdm import tqdm

np.random.seed(42)
plt.rcParams["font.family"] = "Arial"

In [2]:
# data
df_cov = pd.read_csv("data/cov.tsv", sep="\t", index_col=0).reset_index(drop=True)
n_indiv = df_cov.shape[0]
df_cov["GIA"] = np.sort(stats.expon.ppf(np.linspace(0.01, 0.4, n_indiv)))[
    stats.rankdata(df_cov["PC1"], method="ordinal") - 1
]
df_cov.drop(columns="PC1", inplace=True)

df_cov = (df_cov - df_cov.mean(axis=0)) / df_cov.std(axis=0)

df_cov = df_cov.sample(n=50000)

# parameter
baseline_r2 = 0.25
cov_effects = [0.25, 0.2, 0.15]

# simulate data
cov = df_cov.values
n_indiv = cov.shape[0]
pred = np.random.normal(size=n_indiv)
design = np.hstack([np.ones((n_indiv, 1)), pred.reshape(-1, 1), cov])

true_beta = np.array([0, 1] + [0] * cov.shape[1])
true_gamma = np.array([np.log(1 / baseline_r2 - 1), 0] + list(cov_effects))
slope = np.ones(n_indiv)
slope[cov[:, 1] > 0] += 5
y = np.random.normal(
    loc=(design @ true_beta) * slope, scale=np.sqrt(np.exp(design @ true_gamma))
)

# prepare data
mean_covar, var_covar, y = design, design, y
slope_covar = cov.copy()
slope_covar[:, 1] = (slope_covar[:, 1] > 0).astype(float)

In [3]:
# female = cov[:, 1] < 0
# fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
# plt.scatter(pred[female], y[female], s=2, label="Female")
# plt.scatter(pred[~female], y[~female], s=2, label="Male")
# plt.legend()
# plt.show()

In [4]:
def fit_het_linear_vary_slope(
    y: np.ndarray,
    mean_covar: np.ndarray,
    var_covar: np.ndarray,
    slope_covar: np.ndarray,
):
    """Fit `fit_het_linear` with varying slope
    y ~ N((mean_covar * mean_beta) * (1 + slope_covar * slope_beta), exp(var_covar * var_beta))

    """
    slope_beta = np.zeros(slope_covar.shape[1])

    for i in range(500):

        slope_weight = 1 + slope_covar @ slope_beta

        print(f"## iter{i}")
        print("slope_beta: ", slope_beta[0:5])
        print("slope_weight: ", slope_weight[0:5])
        beta, gamma = calpgs.fit_het_linear(
            y=y,
            mean_covar=mean_covar * slope_weight[:, None],
            var_covar=var_covar,
            method="remlscore",
            return_est_covar=False,
        )
        fitted_mu = mean_covar @ beta
        fitted_var = np.exp(var_covar.dot(gamma))
        print("fitted_mu: ", fitted_mu)
        print("fitted_mu: ", fitted_mu)

        # fit slope beta
        #     y ~ N(mu * (1 + slope_covar * slope_beta), var)
        # ==> y - mu ~ N(mu * slope_covar * slope_beta, var)
        slope_beta = (
            sm.WLS(
                endog=y - fitted_mu,
                exog=fitted_mu[:, None] * slope_covar,
                weights=1.0 / fitted_var,
            )
            .fit()
            .params
        )